## @field_validator in Pydantic


#### 1. field_validator Simple Custom Logic


In [1]:
from pydantic import BaseModel, field_validator
from typing import ClassVar


class UserPreferences(BaseModel):
    # Field to be validated
    theme_color: str
    font_size: int

    # Class variable for validation (not a model field)
    VALID_COLORS: ClassVar[list[str]] = ["dark", "light", "blue"]

    # 1. Define the validator function
    # @field_validator('<field_name>') specifies which field to validate
    # 'cls' is the class, 'v' is the value received for the field
    @field_validator("theme_color")
    @classmethod
    def check_valid_color(cls, v: str) -> str:
        # 2. Implement custom validation logic
        v_lower = v.lower()
        if v_lower not in cls.VALID_COLORS:
            raise ValueError(f"Invalid color '{v}'. Must be one of: {', '.join(cls.VALID_COLORS)}")
        # 3. Always return the (possibly transformed) value
        return v_lower

In [2]:
# Valid usage
user1 = UserPreferences(theme_color="Dark", font_size=12)
print(f"User 1 Color (Transformed): {user1.theme_color}")

User 1 Color (Transformed): dark


In [3]:
# Invalid usage
UserPreferences(theme_color="Green", font_size=14)

ValidationError: 1 validation error for UserPreferences
theme_color
  Value error, Invalid color 'Green'. Must be one of: dark, light, blue [type=value_error, input_value='Green', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

---

#### 2. field_validator Using mode='before'


In [4]:
from pydantic import BaseModel, field_validator
import re


class PhoneNumber(BaseModel):
    phone_number: str

    # Use mode='before' to clean the string before Pydantic validates it
    @field_validator("phone_number", mode="before")
    @classmethod
    def clean_phone_number(cls, v: str):
        if not isinstance(v, str):
            raise ValueError("Phone number must be str")

        # 1. Remove non-digit characters (spaces, hyphens, parentheses)
        cleaned_v = re.sub(r"\D", "", v)

        # 2. Pre-validation: Check for minimum length after cleaning
        if len(cleaned_v) < 10:
            raise ValueError("Phone number must contain at least 10 digits.")

        # 3. Return the cleaned string
        return cleaned_v

In [5]:
# Input data is messy but is cleaned and validated
phone_data = {"phone_number": "(555) 123-4567"}
phone1 = PhoneNumber(**phone_data)
print(f"Cleaned Phone: {phone1.phone_number}")

Cleaned Phone: 5551234567


In [6]:
# Invalid
PhoneNumber(phone_number="123-456")

ValidationError: 1 validation error for PhoneNumber
phone_number
  Value error, Phone number must contain at least 10 digits. [type=value_error, input_value='123-456', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

In [7]:
# Invalid
PhoneNumber(phone_number=1234567890)

ValidationError: 1 validation error for PhoneNumber
phone_number
  Value error, Phone number must be str [type=value_error, input_value=1234567890, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

---

#### 3. field_validator Using mode='after'


In [8]:
from pydantic import BaseModel, field_validator


class InventoryItem(BaseModel):
    # Pydantic will coerce the value to an int first
    stock_level: int

    # mode='after' (default) means 'v' is guaranteed to be an int (or fail validation)
    @field_validator("stock_level", mode="after")
    @classmethod
    def must_be_positive(cls, v: int) -> int:
        if v < 0:
            raise ValueError("Stock level cannot be negative.")
        return v

In [9]:
item1 = InventoryItem(stock_level="10")
print(f"Valid Item Stock: {item1.stock_level}")

Valid Item Stock: 10


In [10]:
InventoryItem(stock_level="-5")

ValidationError: 1 validation error for InventoryItem
stock_level
  Value error, Stock level cannot be negative. [type=value_error, input_value='-5', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

---

## @model_validator in Pydantic


#### 1. model_validator Using mode='before'


In [11]:
from pydantic import BaseModel, model_validator
from typing import Any


class Person(BaseModel):
    # Define the expected fields of the model
    name: str
    age: int

    @model_validator(mode="before")  # Runs before Pydantic parses/validates fields
    def from_list(data: Any):
        # Normalize the name: remove surrounding spaces and convert to Title Case
        data["name"] = data["name"].strip().title()

        # Validate 'age' before Pydantic processes it
        if data["age"] < 0:
            raise ValueError("Age should be greater than zero")

        # Return the modified data back to Pydantic
        return data

In [12]:
# Valid
p1 = Person(name="yash", age=25)
print(p1)

name='Yash' age=25


In [13]:
# Invalid
p2 = Person(name="yash", age=-1)
print(p2)

ValidationError: 1 validation error for Person
  Value error, Age should be greater than zero [type=value_error, input_value={'name': 'Yash', 'age': -1}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

---

#### 2. model_validator Using mode='after'


In [14]:
from pydantic import BaseModel, model_validator
from typing import Optional


class DateRange(BaseModel):
    start_date: str
    end_date: Optional[str] = None

    # 1. Define the model validator function
    # @model_validator(mode='after') means it runs on the final model instance (self)
    @model_validator(mode="after")
    def check_dates_cohesion(self):
        # 2. Get the values of the fields from 'self'
        start = self.start_date
        end = self.end_date

        # 3. Implement cross-field logic
        # Assuming simple string comparison for demonstration, but imagine date objects here:
        if end and (end < start):
            # Raise ValueError if the end date is before the start date
            raise ValueError("End date cannot be before the start date.")

        # 4. Always return 'self' at the end of an 'after' model_validator
        return self

In [15]:
# Valid date range
range1 = DateRange(start_date="2024-01-10", end_date="2024-01-20")
print(f"Valid Range: {range1.start_date} to {range1.end_date}")

Valid Range: 2024-01-10 to 2024-01-20


In [16]:
# Invalid
DateRange(start_date="2024-02-01", end_date="2024-01-01")

ValidationError: 1 validation error for DateRange
  Value error, End date cannot be before the start date. [type=value_error, input_value={'start_date': '2024-02-0...end_date': '2024-01-01'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error